In [1]:
import numpy as np
from baselines import ZeroBaselineFactory
from evaluators import ProportionalityEvaluator
from helpers import load_albert_v2, load_imdb_albert_lig_data, extract_token_ids_and_attributions, wrap_call_with_numpy
from attribution_methods import RandomAttributionValues
import wandb
from tqdm import tqdm

In [2]:
wandb.init(project="my-test-project", entity="tpatzelt")

wandb: Currently logged in as: tpatzelt (use `wandb login --relogin` to force relogin)


In [3]:
data = load_imdb_albert_lig_data()
model = load_albert_v2()
callable_model = wrap_call_with_numpy(model)

observation, attribution_values = extract_token_ids_and_attributions(data[0])

Reusing dataset thermostat (/home/tim/.cache/huggingface/datasets/thermostat/imdb-albert-lig/1.0.1/0cbe93e1fbe5b8ed0217559442d8b49a80fd4c2787185f2d7940817c67d8707b)


Loading Thermostat configuration: imdb-albert-lig


In [4]:
evaluator = ProportionalityEvaluator(model=callable_model, baseline_factory=ZeroBaselineFactory)

In [5]:
tpn_lig = evaluator.compute_tpn(observation=observation, attribution_values=attribution_values)
tps_lig = evaluator.compute_tps(observation=observation, attribution_values=attribution_values)
print(f"{tpn_lig = }")
print(f"{tps_lig = }")

tpn_lig = 0.000103822297676191
tps_lig = 0.00021412324486238981


In [6]:
tpn_results, tps_results = [], []
for i in tqdm(range(1)):
    random_attribution_values = RandomAttributionValues.get_attribution_values(observation=observation)
    tpn = evaluator.compute_tpn(observation=observation, attribution_values=random_attribution_values)
    tps = evaluator.compute_tps(observation=observation, attribution_values=random_attribution_values)
    tpn_results.append(tpn)
    tps_results.append(tps)



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.28s/it]


In [7]:
print(np.mean(tpn_results))
print(np.mean(tps_results))

0.0002525460910162743
0.0004933038715275941


In [10]:
data = [[x, y] for (x, y) in zip([1,2,3], [1,2,4])]
table = wandb.Table(data=data, columns = ["x", "y"])
wandb.log({"my_custom_plot_id" : wandb.plot.line(table, "x", "y",
           title="Custom Y vs X Line Plot")})

wandb: Network error (ConnectionError), entering retry loop.
